In [1]:
# | default_exp nets/perceiver_3d

# Imports

In [2]:
# | export

from typing import Literal

import torch
from einops import rearrange, repeat
from huggingface_hub import PyTorchModelHubMixin
from pydantic import BaseModel
from torch import nn

from vision_architectures.layers.attention import Attention3DLayer
from vision_architectures.layers.embeddings import AbsolutePositionEmbeddings3D

# Config

In [3]:
# | export


class Perceiver3DEncoderConfig(BaseModel):
    in_channels: int

    dim: int
    num_latent_tokens: int

    num_encode_layers: int
    num_process_layers: int

    num_encode_cross_attention_heads: int
    num_process_self_attention_heads: int

    encode_cross_attention_kwargs: dict = {}
    process_self_attention_kwargs: dict = {}


class Perceiver3DDecoderConfig(BaseModel):
    out_channels: int
    out_shape: tuple[int, int, int]

    dim: int

    position_embeddings: Literal["absolute"] | None = "absolute"
    position_embeddings_kwargs: dict = {}

    num_decode_layers: int
    num_decode_cross_attention_heads: int
    decode_cross_attention_kwargs: dict = {}


class Perceiver3DConfig(Perceiver3DEncoderConfig, Perceiver3DDecoderConfig):
    pass

In [4]:
test_config = Perceiver3DConfig.model_validate(
    {
        "in_channels": 1,
        "out_channels": 1,
        "out_shape": (32, 32, 32),
        "dim": 384,
        "num_latent_tokens": 1024,
        "num_encode_layers": 1,
        "num_process_layers": 3,
        "num_decode_layers": 3,
        "num_encode_cross_attention_heads": 16,
        "num_process_self_attention_heads": 16,
        "num_decode_cross_attention_heads": 16,
    }
)

# Architecture

### Encoder

In [5]:
# | export


class Perceiver3DEncoderEncode(nn.Module):
    def __init__(
        self,
        in_channels: int,
        dim: int,
        num_tokens: int,
        num_attention_heads: int,
        num_layers: int,
        **cross_attention_kwargs
    ):
        super().__init__()

        self.channel_mapping = nn.Conv3d(in_channels, dim, kernel_size=1, stride=1)
        self.latent_tokens = nn.Parameter(torch.empty(num_tokens, dim), requires_grad=True)
        nn.init.xavier_uniform_(self.latent_tokens)

        self.cross_attention = nn.ModuleList(
            [Attention3DLayer(dim, num_attention_heads, **cross_attention_kwargs) for _ in range(num_layers)]
        )

    def forward(self, x, return_all: bool = False) -> torch.Tensor | dict[str, torch.Tensor]:
        # x: (b, in_channels, z, y, x)

        x = self.channel_mapping(x)
        # (b, dim, z, y, x)

        b = x.shape[0]

        q = repeat(self.latent_tokens, "t d -> b t d", b=b)
        # (b, num_tokens, dim)
        kv = rearrange(x, "b d z y x -> b (z y x) d")
        # (b, z*y*x, dim)
        embeddings = [q]
        for cross_attention_layer in self.cross_attention:
            q = embeddings[-1]
            embeddings.append(cross_attention_layer(q, kv, kv, tokens_as_3d=False))
        # (b, num_tokens, dim)

        return_value = embeddings[-1]
        if return_all:
            return_value = {
                "embeddings": return_value,
                "all_embeddings": embeddings,
            }

        return return_value

In [6]:
test = Perceiver3DEncoderEncode(1, 512, 1024, 16, 1)
display(test)
o = test(torch.randn(2, 1, 32, 32, 32))
display(o.shape)


Perceiver3DEncoderEncode(
  (channel_mapping): Conv3d(1, 512, kernel_size=(1, 1, 1), stride=(1, 1, 1))
  (cross_attention): ModuleList(
    (0): Attention3DLayer(
      (attn): MultiHeadAttention3D(
        (W_q): Linear(in_features=512, out_features=512, bias=True)
        (W_k): Linear(in_features=512, out_features=512, bias=True)
        (W_v): Linear(in_features=512, out_features=512, bias=True)
        (proj): Linear(in_features=512, out_features=512, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (layernorm1): LayerNorm((512,), eps=1e-06, elementwise_affine=True)
      (mlp): Attention3DMLP(
        (dense1): Linear(in_features=512, out_features=2048, bias=True)
        (act): GELU(approximate='none')
        (dense2): Linear(in_features=2048, out_features=512, bias=True)
        (dropout): Dropout(p=0.0, inplace=False)
      )
      (layernorm2): LayerNorm((512,), eps=1e-06, elementwise_affine=True)
    )
  )
)

torch.Size([2, 1024, 512])

In [7]:
# | export


class Perceiver3DEncoderProcess(nn.Module):
    def __init__(self, dim: int, num_attention_heads: int, num_layers: int, **self_attention_kwargs):
        super().__init__()

        self.self_attention = nn.ModuleList(
            [Attention3DLayer(dim, num_attention_heads, **self_attention_kwargs) for _ in range(num_layers)]
        )

    def forward(self, q, return_all: bool = False) -> torch.Tensor | dict[str, torch.Tensor]:
        # q: (b, num_tokens, dim)

        embeddings = [q]
        for self_attention_layer in self.self_attention:
            qkv = embeddings[-1]
            embeddings.append(self_attention_layer(qkv, qkv, qkv, tokens_as_3d=False))
        # (b, num_tokens, dim)

        return_value = embeddings[-1]
        if return_all:
            return_value = {
                "embeddings": return_value,
                "all_embeddings": embeddings,
            }

        return return_value

In [8]:
test = Perceiver3DEncoderProcess(1024, 16, 2)
display(test)
o = test(torch.randn(2, 512, 1024))
display(o.shape)


Perceiver3DEncoderProcess(
  (self_attention): ModuleList(
    (0-1): 2 x Attention3DLayer(
      (attn): MultiHeadAttention3D(
        (W_q): Linear(in_features=1024, out_features=1024, bias=True)
        (W_k): Linear(in_features=1024, out_features=1024, bias=True)
        (W_v): Linear(in_features=1024, out_features=1024, bias=True)
        (proj): Linear(in_features=1024, out_features=1024, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (layernorm1): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
      (mlp): Attention3DMLP(
        (dense1): Linear(in_features=1024, out_features=4096, bias=True)
        (act): GELU(approximate='none')
        (dense2): Linear(in_features=4096, out_features=1024, bias=True)
        (dropout): Dropout(p=0.0, inplace=False)
      )
      (layernorm2): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
    )
  )
)

torch.Size([2, 512, 1024])

In [9]:
# | export


class Perceiver3DEncoder(nn.Module, PyTorchModelHubMixin):
    def __init__(self, config: Perceiver3DEncoderConfig | Perceiver3DConfig):
        super().__init__()

        self.encode = Perceiver3DEncoderEncode(
            config.in_channels,
            config.dim,
            config.num_latent_tokens,
            config.num_encode_cross_attention_heads,
            config.num_encode_layers,
            **config.encode_cross_attention_kwargs,
        )
        self.process = Perceiver3DEncoderProcess(
            config.dim,
            config.num_process_self_attention_heads,
            config.num_process_layers,
            **config.process_self_attention_kwargs,
        )

    def forward(self, x, return_all: bool = False) -> torch.Tensor | dict[str, torch.Tensor]:
        # x: (b, in_channels, z, y, x)

        return_value = {}

        encode_embeddings = self.encode(x, return_all=True)["all_embeddings"]
        return_value["encode_embeddings"] = encode_embeddings
        embeddings = encode_embeddings[-1]
        # (b, num_tokens, dim)

        process_embeddings = self.process(embeddings, return_all=True)["all_embeddings"]
        return_value["process_embeddings"] = process_embeddings
        embeddings = process_embeddings[-1]

        return_value["embeddings"] = embeddings

        if not return_all:
            return_value = embeddings

        return return_value

In [10]:
test = Perceiver3DEncoder(test_config)
display(test)
o = test(torch.randn(2, 1, 24, 24, 24))
display(o.shape)


Perceiver3DEncoder(
  (encode): Perceiver3DEncoderEncode(
    (channel_mapping): Conv3d(1, 384, kernel_size=(1, 1, 1), stride=(1, 1, 1))
    (cross_attention): ModuleList(
      (0): Attention3DLayer(
        (attn): MultiHeadAttention3D(
          (W_q): Linear(in_features=384, out_features=384, bias=True)
          (W_k): Linear(in_features=384, out_features=384, bias=True)
          (W_v): Linear(in_features=384, out_features=384, bias=True)
          (proj): Linear(in_features=384, out_features=384, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (layernorm1): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
        (mlp): Attention3DMLP(
          (dense1): Linear(in_features=384, out_features=1536, bias=True)
          (act): GELU(approximate='none')
          (dense2): Linear(in_features=1536, out_features=384, bias=True)
          (dropout): Dropout(p=0.0, inplace=False)
        )
        (layernorm2): LayerNorm((384,), eps=1e-06, element

torch.Size([2, 1024, 384])

### Decoder

In [11]:
# | export


class Perceiver3DDecoder(nn.Module, PyTorchModelHubMixin):
    def __init__(self, config: Perceiver3DDecoderConfig | Perceiver3DConfig):
        super().__init__()

        self.config = config

        self.empty_token = nn.Parameter(torch.empty(config.dim, 1), requires_grad=True)
        nn.init.xavier_uniform_(self.empty_token)

        self.position_embeddings = None
        if config.position_embeddings == "absolute":
            self.position_embeddings = AbsolutePositionEmbeddings3D(
                config.dim, config.out_shape, **config.position_embeddings_kwargs
            )
        elif config.position_embeddings is None:
            pass
        else:
            raise NotImplementedError(f"Position embeddings {config.position_embeddings} not implemented")

        self.cross_attention = nn.ModuleList(
            [
                Attention3DLayer(
                    config.dim, config.num_decode_cross_attention_heads, **config.decode_cross_attention_kwargs
                )
                for _ in range(config.num_decode_layers)
            ]
        )

        self.channel_mapping = nn.Conv3d(config.dim, config.out_channels, kernel_size=1, stride=1)

    def forward(self, kv, return_all: bool = False) -> torch.Tensor | dict[str, torch.Tensor]:
        # kv: (b, num_tokens, dim)

        b = kv.shape[0]

        q = repeat(
            self.empty_token,
            "d 1 -> b d z y x",
            b=b,
            z=self.config.out_shape[0],
            y=self.config.out_shape[1],
            x=self.config.out_shape[2],
        )
        # (b, dim, z, y, x)

        if self.position_embeddings is not None:
            q = q + self.position_embeddings(batch_size=b)

        q = rearrange(q, "b d z y x -> b (z y x) d")
        # (b, num_output_tokens, dim)
        outputs = [q]
        for cross_attention_layer in self.cross_attention:
            q = outputs[-1]
            outputs.append(cross_attention_layer(q, kv, kv, tokens_as_3d=False))
        # (b, num_output_tokens, dim)

        output = outputs[-1]
        output = rearrange(
            output,
            "b (z y x) d -> b d z y x",
            z=self.config.out_shape[0],
            y=self.config.out_shape[1],
            x=self.config.out_shape[2],
        )
        # (b, dim, z, y, x)

        output = self.channel_mapping(output)
        # (b, out_channels, z, y, x)

        return_value = output
        if return_all:
            return_value = {
                "output": output,
                "all_outputs": outputs,
            }

        return return_value

In [12]:
test = Perceiver3DDecoder(test_config)
display(test)
o = test(torch.randn(2, 1024, 384))
display(o.shape)


Perceiver3DDecoder(
  (position_embeddings): AbsolutePositionEmbeddings3D()
  (cross_attention): ModuleList(
    (0-2): 3 x Attention3DLayer(
      (attn): MultiHeadAttention3D(
        (W_q): Linear(in_features=384, out_features=384, bias=True)
        (W_k): Linear(in_features=384, out_features=384, bias=True)
        (W_v): Linear(in_features=384, out_features=384, bias=True)
        (proj): Linear(in_features=384, out_features=384, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (layernorm1): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
      (mlp): Attention3DMLP(
        (dense1): Linear(in_features=384, out_features=1536, bias=True)
        (act): GELU(approximate='none')
        (dense2): Linear(in_features=1536, out_features=384, bias=True)
        (dropout): Dropout(p=0.0, inplace=False)
      )
      (layernorm2): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
    )
  )
  (channel_mapping): Conv3d(384, 1, kernel_size=(1, 1, 1), st

torch.Size([2, 1, 32, 32, 32])

# nbdev

In [13]:
!nbdev_export